In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score,make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVR
from metrics import partsMetrics,allMetrics

In [2]:
df=pd.read_csv('processed.csv')
print('Data shape:',df.shape)

Data shape: (467, 7)


In [3]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values
print(X.shape)
print(y.shape)

(467, 6)
(467,)


In [4]:
C_values=list(np.logspace(-3,3,7)) + [200]
gamma_values=list(np.logspace(-3,3,7)) + ['scale','auto']
param_grid=[
    {
        'kernel': ['rbf','sigmoid','linear'],
        'C': C_values,
        'gamma': gamma_values,
        'epsilon': [1e-2,.1,1],
        'shrinking': [True,False]
    },
    {
        'kernel': ['poly'],
        'C': C_values,
        # 'gamma': gamma_values,
        'degree': [3,4,5],
        'epsilon': [1e-2,.1,1],
        'shrinking': [True,False]
    }
]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

def train_model(X_train,y_train):
    reg = SVR()
    grid=GridSearchCV(reg,param_grid,cv=5,scoring=make_scorer(r2_score),n_jobs=2)
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    print(grid.best_score_)
    return grid

In [74]:
model=train_model(X_train,y_train)

{'C': 200, 'epsilon': 1, 'gamma': 'auto', 'kernel': 'rbf', 'shrinking': True}
0.8460150042405947


In [5]:
best_params={'C': 200, 'epsilon': 1, 'gamma': 'auto', 'kernel': 'rbf', 'shrinking': True}
reg=SVR(**best_params)
partsMetrics(df,reg)

train rmse: 2.0960124407574185
val rmse: 4.453297849827749
test rmse: 4.816731132435866

train si: 0.1924094238759525
val si: 0.372241866981313
test si: 0.46923343536145107

train r2: 0.9580282456216203
val r2: 0.8506740968111035
test r2: 0.8149085381759513

train mape: 76.78495169367272
val mape: 61.97672241646834
test mape: 68.30770958002063


In [6]:
allMetrics(df,reg)

all rmse: 2.3277083479264356
all si: 0.2111738267651204
all r2: 0.9516415366871102
all mape: 62.1320726299999
